# Q2: Data Cleaning

**Phase 3:** Data Cleaning & Preprocessing  
**Points: 9 points**

**Focus:** Handle missing data, outliers, validate data types, remove duplicates.

**Lecture Reference:** Lecture 11, Notebook 1 ([`11/demo/01_setup_exploration_cleaning.ipynb`](https://github.com/christopherseaman/datasci_217/blob/main/11/demo/01_setup_exploration_cleaning.ipynb)), Phase 3. Also see Lecture 05 (data cleaning).

---

## Setup

In [47]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Load data from Q1 (or directly from source)
df = pd.read_csv('data/beach_sensors.csv')
# If you saved cleaned data from Q1, you can load it:
# df = pd.read_csv('output/q1_exploration.csv')  # This won't work - load original

In [48]:
# Check if missing values occur in the same rows
cols_with_same_missing = ['Wet Bulb Temperature', 'Rain Intensity', 'Total Rain', 'Precipitation Type','Heading' ]
missing_mask = df[cols_with_same_missing].isnull()
print(f"Rows where ALL are missing: {missing_mask.all(axis=1).sum():,}")
print(f"Rows where ANY is missing: {missing_mask.any(axis=1).sum():,}")
# Check for missing values
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Missing %': missing_pct.round(2)
})
missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)
display(missing_df)

Rows where ALL are missing: 75,981
Rows where ANY is missing: 75,981


,Missing Count,Missing %
Wet Bulb Temperature,75981,38.69
Rain Intensity,75981,38.69
Total Rain,75981,38.69
Precipitation Type,75981,38.69
Heading,75981,38.69
Barometric Pressure,146,0.07
Air Temperature,75,0.04


In [49]:
#check for duplicates
print(df.shape)
display(df.duplicated().sum()) 
df_clean = df.drop_duplicates()
print(df_clean.shape)
#no duplicates 

#converting types
df['Measurement Timestamp'] = pd.to_datetime(df['Measurement Timestamp'])
display(df.dtypes) 
print(df['Measurement ID'].head(5))

#dealing with outliers

# Solar Radiation cant be greater than 20000 or less than 0
df.loc[(df['Solar Radiation'] < 0) | (df['Solar Radiation'] > 2000), 'Solar Radiation'] = np.nan

# Interval Rain: can't be negative
df.loc[df['Interval Rain'] < 0, 'Interval Rain'] = np.nan

# Wind Speed: replace 999.9 sentinel
df.loc[df['Wind Speed'] > 200, 'Wind Speed'] = np.nan

# Maximum Wind Speed: replace 999.9 sentinel
df.loc[df['Maximum Wind Speed'] > 200, 'Maximum Wind Speed'] = np.nan

# Barometric Pressure: typical 950-1050 hPa
df.loc[(df['Barometric Pressure'] < 900) | (df['Barometric Pressure'] > 1100), 'Barometric Pressure'] = np.nan

# Summary of cleaning
print("\n Data Cleaning Summary")
print(f"Solar Radiation missing: {df['Solar Radiation'].isna().sum()}")
print(f"Interval Rain missing: {df['Interval Rain'].isna().sum()}")
print(f"Wind Speed missing: {df['Wind Speed'].isna().sum()}")
print(f"Max Wind Speed missing: {df['Maximum Wind Speed'].isna().sum()}")
print(f"Barometric Pressure missing: {df['Barometric Pressure'].isna().sum()}")

# New statistics after cleaning
print("\nCleaned Data Statistics")
display(df[['Solar Radiation', 'Interval Rain', 'Wind Speed', 'Maximum Wind Speed', 'Barometric Pressure']].describe())

(196381, 18)


np.int64(0)

(196381, 18)


Station Name                           object
Measurement Timestamp          datetime64[ns]
Air Temperature                       float64
Wet Bulb Temperature                  float64
Humidity                                int64
Rain Intensity                        float64
Interval Rain                         float64
Total Rain                            float64
Precipitation Type                    float64
Wind Direction                          int64
Wind Speed                            float64
Maximum Wind Speed                    float64
Barometric Pressure                   float64
Solar Radiation                         int64
Heading                               float64
Battery Life                          float64
Measurement Timestamp Label            object
Measurement ID                         object
dtype: object

0    63rdStreetWeatherStation201809271000
1    63rdStreetWeatherStation201809271100
2    63rdStreetWeatherStation201809271300
3        FosterWeatherStation201809271300
4    63rdStreetWeatherStation201809271500
Name: Measurement ID, dtype: object

 Data Cleaning Summary
Solar Radiation missing: 13425
Interval Rain missing: 1
Wind Speed missing: 5
Max Wind Speed missing: 6
Barometric Pressure missing: 153

Cleaned Data Statistics


,Solar Radiation,Interval Rain,Wind Speed,Maximum Wind Speed,Barometric Pressure
count,182956.000000,196380.000000,196376.000000,196375.000000,196228.000000
mean,127.921413,0.142324,2.893240,3.528696,994.335504
std,220.928031,1.096737,1.793957,3.030640,6.912603
min,0.000000,0.000000,0.000000,0.000000,958.700000
25%,0.000000,0.000000,1.600000,1.200000,990.200000
50%,5.000000,0.000000,2.900000,3.100000,994.400000
75%,154.000000,0.000000,3.300000,5.200000,998.600000
max,1277.000000,63.420000,39.000000,26.300000,1022.700000


In [50]:
#handling missing values
# Check for missing values
missing1 = df.isnull().sum()
missing_pct1 = (missing / len(df)) * 100
missing_df1 = pd.DataFrame({
    'Missing Count': missing1,
    'Missing %': missing_pct1.round(2)
})
missing_df1 = missing_df1[missing_df1['Missing Count'] > 0].sort_values('Missing Count', ascending=False)
display(missing_df1)#is empty now because filled all missing values


#these values go to 0 because if ther is no rain then these cant be filled out which is why they are missing 
cols_0fill = ['Rain Intensity', 'Total Rain', 'Interval Rain', 'Precipitation Type', 'Wet Bulb Temperature']
df[cols_0fill] = df[cols_0fill].fillna(0)

#these values will be ffill because they are likely similar to the previous measured
col_ffill = ['Wind Speed', 'Maximum Wind Speed', 'Barometric Pressure', 'Interval Rain', 'Heading', 'Air Temperature']
df[col_ffill] = df[col_ffill].ffill()
#these values will be mean filled 
col_meanfill = ['Solar Radiation']
df[col_meanfill] = df[col_meanfill].fillna(df[col_meanfill].mean())

df.to_csv('output/q2_cleaned_data.csv', index = False)


missing2 = df.isnull().sum()
missing_pct2 = (missing / len(df)) * 100
missing_df2 = pd.DataFrame({
    'Missing Count': missing2,
    'Missing %': missing_pct2.round(2)
})
missing_df2 = missing_df2[missing_df2['Missing Count'] > 0].sort_values('Missing Count', ascending=False)
display(missing_df2)

,Missing Count,Missing %
Wet Bulb Temperature,75981,38.69
Rain Intensity,75981,38.69
Total Rain,75981,38.69
Precipitation Type,75981,38.69
Heading,75981,38.69
Solar Radiation,13425,0.00
Barometric Pressure,153,0.07
Air Temperature,75,0.04
Maximum Wind Speed,6,0.00
Wind Speed,5,0.00


,Missing Count,Missing %


In [ ]:
#write report 
missing_info = [
        ("Air Temperature", 75, 0.04, "Forward Fill, likely similar to previous measurement"),
        ("Barometric Pressure", 146, 0.07, "Forward Fill, likely similar to previous measurement"),
        ("Rain Intensity", 75981, 38.69, "Filled with 0 (absence of rain is expected)"),
        ("Total Rain", 75981, 38.69, "Filled with 0 (absence of rain is expected)"),
        ("Interval Rain", 75981, 38.69, "Filled with 0 (absence of rain is expected)"),
        ("Precipitation Type", 75981, 38.69, "Filled with 0 (no precipitation)"),
        ("Wet Bulb Temperature", 75981, 38.69, "Filled with 0 (no rain no wet bulb)"),
        ("Heading", 75981, 38.69, "Forward Fill, likely similar to previous measurement"),
        ("Wind Speed", 5, 0.00, "Forward Fill, likley similar to previous measurement"),
        ("Maximum Wind Speed", 5, 0.00, "Forward Fill, likley similar to previous measurement"),
        ("Solar Radiation", 13425, 0.00, "Mean, there was too many to forward fill so hope mean will suffice"),
    
    ]

outliers = [
        ("Solar Radiation", "< 0", "Replaced with NaN, then interpolated", "Negative values including -100000"),
        ("Interval Rain", "< 0", "Replaced with NaN, then filled with 0", "Negative values"),
        ("Wind Speed", "> 200 mph", "Replaced with NaN, then interpolated", "Sentinel value 999.9"),
        ("Maximum Wind Speed", "> 200 mph", "Replaced with NaN, then interpolated", "Sentinel value 999.9"),
        ("Barometric Pressure", "outside [900, 1100] hPa", "Replaced with NaN, then interpolated", "Values 0.0 and 3098.5")
    ]




with open("output/q2_cleaning_report.txt", 'w') as f:
    f.write(f'DATA CLEANING REPORT\n')
    f.write(f'====================\n')
    f.write(f'Rows Before Cleaning:{len(df)}\n')
    f.write(f'====================\n')
    f.write("Data Type Conversions:\n")
    f.write("- Measurement Timestamp: Converted from object to datetime64[ns]\n\n")
    f.write(f'====================\n')
    f.write(f"Duplicates Removed:0\n")
    f.write(f'====================\n')
    f.write(f'Handling Missing Data:\n')
    for col, count, pct, method in missing_info:
        f.write(f"- {col}: {count} missing values\n")
        f.write(f"  Method: {method}\n")
        f.write(f"  Result: All missing values filled\n\n")
    f.write(f'====================\n')
    f.write("Outlier Handling:\n\n")
    for col, condition, method, details in outliers:
        f.write(f"- {col}: Detected values {condition}\n")
        f.write(f"  Method: {method}\n")
        f.write(f"  Invalid values: {details}\n")
        f.write(f"  Result: Invalid values corrected\n\n")
    f.write(f'====================\n')
    f.write(f'Rows after cleaning: {len(df)}\n')
    f.write(f'ALL MISSING DATA WAS HANDLED')


In [54]:
with open('output/q2_rows_cleaned.txt', 'w') as f:
    f.write(f'{len(df)}')

---

## Objective

Clean the dataset by handling missing data, outliers, validating data types, and removing duplicates.

**Time Series Note:** For time series data, forward-fill (`ffill()`) is often appropriate for missing values since sensor readings are continuous. However, you may choose other strategies based on your analysis.

---

## Required Artifacts

You must create exactly these 3 files in the `output/` directory:

### 1. `output/q2_cleaned_data.csv`
**Format:** CSV file
**Content:** Cleaned dataset with same structure as original (same columns)
**Requirements:**
- Same columns as original dataset
- Missing values handled (filled, dropped, or imputed)
- Outliers handled (removed, capped, or transformed)
- Data types validated and converted
- Duplicates removed
- **Sanity check:** Dataset should retain most rows after cleaning (at least 1,000 rows). If you're removing more than 50% of data, reconsider your strategy—imputation is usually preferable to dropping rows for this dataset.
- **No index column** (save with `index=False`)

### 2. `output/q2_cleaning_report.txt`
**Format:** Plain text file
**Content:** Detailed report of cleaning operations
**Required information:**
- Rows before cleaning: [number]
- Missing data handling method: [description]
  - Which columns had missing data
  - Method used (drop, forward-fill, impute, etc.)
  - Number of values handled
- Outlier handling: [description]
  - Detection method (IQR, z-scores, domain knowledge)
  - Which columns had outliers
  - Method used (remove, cap, transform)
  - Number of outliers handled
- Duplicates removed: [number]
- Data type conversions: [list any conversions]
- Rows after cleaning: [number]

**Example format:**
```
DATA CLEANING REPORT
====================

Rows before cleaning: 50000

Missing Data Handling:
- Water Temperature: 2500 missing values (5.0%)
  Method: Forward-fill (time series appropriate)
  Result: All missing values filled
  
- Air Temperature: 1500 missing values (3.0%)
  Method: Forward-fill, then median imputation for remaining
  Result: All missing values filled

Outlier Handling:
- Water Temperature: Detected 500 outliers using IQR method (3×IQR)
  Method: Capped at bounds [Q1 - 3×IQR, Q3 + 3×IQR]
  Bounds: [-5.2, 35.8]
  Result: 500 values capped

Duplicates Removed: 0

Data Type Conversions:
- Measurement Timestamp: Converted to datetime64[ns]

Rows after cleaning: 50000
```

### 3. `output/q2_rows_cleaned.txt`
**Format:** Plain text file
**Content:** Single integer number (total rows after cleaning)
**Requirements:**
- Only the number, no text, no labels
- No whitespace before or after
- Example: `50000`

---

## Requirements Checklist

- [ ] Missing data handling strategy chosen and implemented
- [ ] Outliers detected and handled (IQR method, z-scores, or domain knowledge)
- [ ] Data types validated and converted
- [ ] Duplicates identified and removed
- [ ] Cleaning decisions documented in report
- [ ] All 3 required artifacts saved with exact filenames

---

## Your Approach

1. **Handle missing data** - Choose appropriate strategy (drop, forward-fill, impute) based on data characteristics
2. **Detect and handle outliers** - Use IQR method or z-scores; decide whether to remove, cap, or transform
3. **Validate data types** - Ensure numeric and datetime columns are properly typed
4. **Remove duplicates**
5. **Document and save** - Write detailed cleaning report explaining your decisions

---

## Decision Points

- **Missing data:** Should you drop rows, impute values, or forward-fill? Consider: How much data is missing? Is it random or systematic? For time series, forward-fill is often appropriate.
- **Outliers:** Are they errors or valid extreme values? Use IQR method or z-scores to detect, then decide: remove, cap, or transform. Document your reasoning.
- **Data types:** Are numeric columns actually numeric? Are datetime columns properly formatted? Convert as needed.

---

## Checkpoint

After Q2, you should have:
- [ ] Missing data handled
- [ ] Outliers addressed
- [ ] Data types validated
- [ ] Duplicates removed
- [ ] All 3 artifacts saved: `q2_cleaned_data.csv`, `q2_cleaning_report.txt`, `q2_rows_cleaned.txt`

---

**Next:** Continue to `q3_data_wrangling.md` for Data Wrangling.
